## 多线程访问同一个列表导致冲突的问题

两个线程同时往一个 List 中加入`MAX_COUNT`个数，最终 List 的数量有可能并非是`2 * MAX_COUNT`


In [11]:
#load "LoadBaseToolWPF.csx"
using BaseTool;
using System.Threading;
using System.IO;

const int MAX_COUNT = 10;
int _count = 0;
while (_count < MAX_COUNT)
{
    List<int> list = new List<int>();

    Task _task1 = Task.Run(() =>
    {
        for (int i = 0; i < MAX_COUNT; i++) { list.Add(i); }
    });

    Task _task2 = Task.Run(() =>
    {
        for (int i = MAX_COUNT; i < 2 * MAX_COUNT; i++) { list.Add(i); }
    });

    Task.WaitAll(_task1, _task2);

    // 当长度对不上时，输出详细内容
    Console.WriteLine($"{_count++}:数量{list.Count}");
    if (list.Count != MAX_COUNT * 2) { list.Dump(); }
    Thread.Sleep(1000);
}

0:数量20
1:数量10
10, 11, 12, 13, 14, 15, 16, 17, 18, 19
2:数量18
10, 11, 12, 13, 14, 15, 16, 17, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9
3:数量20
4:数量20
5:数量20
6:数量20
7:数量20
8:数量20
9:数量20


## 测试直接运行异步函数，而不使用 await 关键字的运行情况


In [12]:
using System.Diagnostics;

Stopwatch _sw = Stopwatch.StartNew();
await Function();
_sw.Stop();
Console.WriteLine($"运行耗时：{_sw.ElapsedMilliseconds}ms");

// 这是一个异步函数，如果不直接执行，则会直接跳过
public async Task Function()
{
    await Task.Delay(2000);
    Console.WriteLine("Hello World");
}

Hello World
运行耗时：2000ms


In [13]:
using System.Diagnostics;

Stopwatch _sw = Stopwatch.StartNew();
Function();
_sw.Stop();
Console.WriteLine($"运行耗时：{_sw.ElapsedMilliseconds}ms");

运行耗时：0ms


## 多线程打断等待测试

- 任务 1 持续循环，等待任务 2 打断后，再过 1 秒，结束。
- 任务 2 等待 2 秒，打断任务 1，等待任务 1 完全结束，也结束。

这里有个问题：
如果任务 2 中没有`await _task`，即取消后就结束任务 2，那么任务 1 最终就不会执行。


In [14]:
using System.Threading.Tasks;

CancellationTokenSource _cts = new CancellationTokenSource();
Task _task = Task.Run(()=>
{
    while(true)
    {
        if(_cts.IsCancellationRequested) break;
    }
    Task.Delay(1000).Wait();
    Console.WriteLine("任务1结束");
});

await Task.Run(async ()=>
{
    Task.Delay(2000).Wait();
    _cts.Cancel();
    await _task;
    Console.WriteLine("任务2结束");
});

任务1结束
任务2结束


## `TaskCompletionSource` 相关状态


In [15]:
TaskCompletionSource _source1 = new TaskCompletionSource();
Console.WriteLine($"初始状态: {_source1.Task.Status}");

_source1.TrySetCanceled();
Console.WriteLine($"TrySetCanceled: {_source1.Task.Status}");

TaskCompletionSource _source2 = new TaskCompletionSource();
_source2.TrySetResult();
Console.WriteLine($"TrySetResult: {_source2.Task.Status}");


TaskCompletionSource _source3 = new TaskCompletionSource();
_source3.TrySetException(new Exception());
Console.WriteLine($"TrySetException: {_source3.Task.Status}");

初始状态: WaitingForActivation
TrySetCanceled: Canceled
TrySetResult: RanToCompletion
TrySetException: Faulted


## `Task` 状态


In [16]:
using System.Threading;

CancellationTokenSource _source = new();
_source.Cancel();
Console.WriteLine($"Task运行: {Task.Run(()=>{}).Status}");
Console.WriteLine($"Task取消: {Task.FromCanceled(_source.Token).Status}");
Console.WriteLine($"Task异常: {Task.FromException(new Exception()).Status}");

Task运行: WaitingToRun
Task取消: Canceled
Task异常: Faulted


## 不能理解的地方


In [17]:
Task _task1 = Task.Delay(1000);
Task _task2 = Task.Run(()=>{ Task.Delay(1000).Wait(); });
Task _task3 = new Task(()=>{ Task.Delay(1000).Wait(); });
_task3.Start();

// 任务运行中
Task.Delay(100).Wait();
Console.WriteLine($"Task1: {_task1.Status}");
Console.WriteLine($"Task1: {_task2.Status}");
Console.WriteLine($"Task1: {_task3.Status}");

// 所有的任务完成
Task.Delay(1100).Wait();
Console.WriteLine($"Task1: {_task1.Status}");
Console.WriteLine($"Task1: {_task2.Status}");
Console.WriteLine($"Task1: {_task3.Status}");

Task1: WaitingForActivation
Task1: Running
Task1: Running
Task1: RanToCompletion
Task1: RanToCompletion
Task1: RanToCompletion


In [ ]:
int value = 10 / 3;
value.Display();

int value01 = 10 % 3;
value01.Display();

object _obj = new object();

int _int_value = 0;
_ = Task.Run(() =>
{
    lock (new object())
    {
        for (int i = 0; i < 2000; i++) 
        { 
            _int_value++; 
        };
        Console.WriteLine($"1:{_int_value}");
    }
});

_ = Task.Run(() =>
{
    lock (new object())
    {
        for (int i = 0; i < 2000; i++) { _int_value++; };
        Console.WriteLine($"2:{_int_value}");
    }
});

In [ ]:

using System.Threading;
double _sum = 0;

int _count = 0;
bool _is_running = true;

CancellationTokenSource _cts = new();

try
{
    _ = Task.Run(()=>
    {
        while(_is_running) 
        { 
            Console.WriteLine($"count = {_count}");
            if(_count > 1e8)
            {
                _cts.Cancel();
                throw new InvalidOperationException();  
            } 
            Task.Delay(100).Wait();
        }

        Console.WriteLine("Task Over");
    }).ContinueWith((_)=>
    {
        Console.WriteLine("Task Over on ContinueWith");
        throw new Exception();
    }, TaskContinuationOptions.OnlyOnFaulted);

    for(int i = 1; i < 1e9; i++)
    {
        _cts.Token.ThrowIfCancellationRequested();
        _count ++;
        _sum += (_sum / i + 1);
    }
    _is_running = false;

    Console.WriteLine(_sum);
}
catch
{
    Console.WriteLine("Catch Exception");
}



count = 14427
count = 15790913
count = 31729677
count = 47635554
count = 63453014
count = 79166046
count = 95005539
count = 110614539
Task Over on ContinueWith
Catch Exception


In [4]:
static async Task FireAndForget()
{
    await Task.Delay(2000);
    Console.WriteLine("FireAndForget");
}

var _task = FireAndForget();
Console.WriteLine("Main Over");
Console.WriteLine(_task.Status);
await Task.Delay(1000);
Console.WriteLine(_task.Status);
await Task.Delay(1000);
Console.WriteLine(_task.Status);
await Task.Delay(1000);
Console.WriteLine(_task.Status);


Main Over
WaitingForActivation
WaitingForActivation
WaitingForActivation
FireAndForget
RanToCompletion


In [5]:
static async Task FireAndForget()
{
    await Task.Delay(2000).ConfigureAwait(false);
    Console.WriteLine("FireAndForget");
}

var _task = FireAndForget();
Console.WriteLine("Main Over");
Console.WriteLine(_task.Status);
await Task.Delay(1000);
Console.WriteLine(_task.Status);
await Task.Delay(1000);
Console.WriteLine(_task.Status);
await Task.Delay(1000);
Console.WriteLine(_task.Status);

Main Over
WaitingForActivation
WaitingForActivation
FireAndForget
RanToCompletion
RanToCompletion


In [15]:
Console.WriteLine($"Main 方法开始，当前线程: {Environment.CurrentManagedThreadId}");
_ = Task.Run(FireAndForget);
Console.WriteLine($"Main 方法继续执行，当前线程: {Environment.CurrentManagedThreadId}");
await Task.Delay(5000);

static async Task FireAndForget()
{
    Console.WriteLine($"FireAndForget 开始，当前线程: {Environment.CurrentManagedThreadId}");
    await Task.Delay(2000).ConfigureAwait(false);
    Console.WriteLine($"FireAndForget 中的 Console.WriteLine，当前线程: {Environment.CurrentManagedThreadId}");
}


Main 方法开始，当前线程: 10
Main 方法继续执行，当前线程: 10
FireAndForget 开始，当前线程: 61
FireAndForget 中的 Console.WriteLine，当前线程: 62


In [16]:
Console.WriteLine($"Main 方法开始，当前线程: {Environment.CurrentManagedThreadId}");
_ = Task.Run(FireAndForget);
Console.WriteLine($"Main 方法继续执行，当前线程: {Environment.CurrentManagedThreadId}");
await Task.Delay(5000);

static async Task FireAndForget()
{
    Console.WriteLine($"FireAndForget 开始，当前线程: {Environment.CurrentManagedThreadId}");
    await Task.Delay(2000);
    Console.WriteLine($"FireAndForget 中的 Console.WriteLine，当前线程: {Environment.CurrentManagedThreadId}");
}

Main 方法开始，当前线程: 10
Main 方法继续执行，当前线程: 10
FireAndForget 开始，当前线程: 61
FireAndForget 中的 Console.WriteLine，当前线程: 37


In [22]:
Console.WriteLine($"Main 方法开始，当前线程: {Environment.CurrentManagedThreadId}");
_ = FireAndForget();
Console.WriteLine($"Main 方法继续执行，当前线程: {Environment.CurrentManagedThreadId}");
await Task.Delay(5000);

static async Task FireAndForget()
{
    Console.WriteLine($"FireAndForget 开始，当前线程: {Environment.CurrentManagedThreadId}");
    await Task.Delay(2000);
    Console.WriteLine($"FireAndForget 中的 Console.WriteLine，当前线程: {Environment.CurrentManagedThreadId}");
}

Main 方法开始，当前线程: 10
FireAndForget 开始，当前线程: 10
Main 方法继续执行，当前线程: 10
FireAndForget 中的 Console.WriteLine，当前线程: 37


In [21]:
Console.WriteLine($"Main 方法开始，当前线程: {Environment.CurrentManagedThreadId}");
_ = FireAndForget();
Console.WriteLine($"Main 方法继续执行，当前线程: {Environment.CurrentManagedThreadId}");
await Task.Delay(5000);

static async Task FireAndForget()
{
    Console.WriteLine($"FireAndForget 开始，当前线程: {Environment.CurrentManagedThreadId}");
    await Task.Delay(2000).ConfigureAwait(false);
    Console.WriteLine($"FireAndForget 中的 Console.WriteLine，当前线程: {Environment.CurrentManagedThreadId}");
}

Main 方法开始，当前线程: 10
FireAndForget 开始，当前线程: 10
Main 方法继续执行，当前线程: 10
FireAndForget 中的 Console.WriteLine，当前线程: 29
